In [71]:
import sys
import os
from imp import reload
from image import utils
import numpy as np
import utils; reload(utils)

<module 'utils' from '../../ml_utils/image/utils.py'>

In [29]:
path = r'../../../data/dbi/'
batch_size=64
# os.mkdir(path+'results')

In [70]:
sys.path.insert(0, '../../ml_utils')
sys.path.insert(0, '../../ml_utils/image')

In [30]:
# trn = utils.get_data(path + r'raw/trn')
# val = utils.get_data(path + r'raw/valid')
# test = utils.get_data(path + r'test')
# utils.save_array(path+'results/trn.dat', trn)
# utils.save_array(path+'results/val.dat', val)
# utils.save_array(path+'results/test.dat', test)

In [36]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = utils.get_classes(path)

Found 9403 images belonging to 120 classes.
Found 819 images belonging to 120 classes.
Found 10357 images belonging to 1 classes.


In [42]:
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

In [63]:
# os.mkdir(path+'sample')
# os.mkdir(path+'sample/train')
# os.mkdir(path+'sample/valid')

# from shutil import copyfile
# from glob import glob

# g = glob(path+'raw/trn/*')
# for d in g: 
#     d = d.split('/')[-1]
#     os.mkdir(path+'sample/train/'+d)
#     os.mkdir(path+'sample/valid/'+d)
# g = glob(path+'raw/trn/*/*.jpg')
# shuf = np.random.permutation(g)
# for i in range(400): 
#     copyfile(shuf[i], path+'sample/train/' + shuf[i].split('/')[-1])
# g = glob(path+'raw/valid/*/*.jpg')
# shuf = np.random.permutation(g)
# for i in range(200): 
#     copyfile(shuf[i], path+'sample/valid/' + shuf[i].split('/')[-1])

In [87]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import BatchNormalization, Convolution2D, MaxPool2D, Dropout, GlobalAveragePooling2D, Activation
from keras.layers import MaxPooling2D
from keras.models import Sequential
import numpy as np

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(244,244,3))

In [88]:
def get_lrg_layers(input_layers):
    nf=128; p=0.
    return [
        BatchNormalization(axis=1, input_shape=input_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(8,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

model.layers

In [94]:
import pickle as pkl

In [95]:
result = pkl.load(open('result.pkl', 'rb'))

In [97]:
result.shape

(10357, 120)

In [98]:
trn_classes

array([  0,   0,   0, ..., 119, 119, 119], dtype=int32)

In [101]:
import pandas as pd

In [104]:
sub = pd.read_csv(path+'sample_submission.csv')


In [108]:
batches = utils.get_batches(path+'raw/trn', batch_size=batch_size)


classes = sorted(batches.class_indices, key=batches.class_indices.get)

Found 9403 images belonging to 120 classes.


In [133]:
submission = pd.DataFrame(result, columns=classes)
submission.insert(0, 'id', [filename.split('.')[0] for filename in raw_test_filenames])
submission.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,2.680094e-08,8.238529e-09,8.865422e-07,3.089517e-07,9.470116e-11,2.809835e-07,5.128263e-06,8.705688e-09,1.003568e-08,...,6.057783e-09,6.015022e-09,8.140311e-09,1.651822e-09,2.712904e-10,1.498201e-08,1.261772e-10,1.240872e-10,1.023233e-07,3.402207e-09
1,00102ee9d8eb90812350685311fe5890,4.409035e-12,4.395454e-10,9.466591e-11,3.499728e-09,1.288064e-10,6.978071e-07,2.637466e-11,2.418920e-09,8.768275e-13,...,1.382480e-13,9.048941e-09,3.611206e-13,4.820020e-13,7.946374e-12,8.536657e-11,4.529498e-11,3.308734e-12,2.928139e-09,1.507963e-11
2,0012a730dfa437f5f3613fb75efcd4ce,1.441759e-07,6.407541e-10,1.719858e-06,1.056395e-09,5.077417e-01,2.035762e-08,7.594557e-08,8.651542e-07,3.030113e-08,...,2.485454e-11,5.811656e-09,9.306422e-06,2.945159e-07,9.096822e-08,2.985240e-09,4.609175e-08,1.343823e-07,8.142599e-09,1.794899e-08
3,001510bc8570bbeee98c8d80c8a95ec1,2.601991e-06,4.635281e-03,3.410056e-06,2.120565e-08,2.879387e-08,1.404188e-10,1.659390e-07,2.657265e-11,7.763093e-09,...,1.327743e-06,1.685799e-08,1.581223e-08,1.527565e-09,5.648541e-09,1.954087e-06,8.913732e-09,1.217109e-07,1.857164e-09,1.923937e-04
4,001a5f3114548acdefa3d4da05474c2e,2.367226e-04,1.720187e-06,7.717064e-07,1.403709e-03,6.398626e-10,1.526640e-06,1.890648e-04,2.548920e-07,5.499503e-08,...,9.028528e-04,2.481087e-04,8.593167e-09,1.573502e-08,1.239299e-08,4.512107e-05,5.253953e-06,6.223127e-09,4.301658e-05,1.946347e-02


In [128]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx)
subm = do_clip(result,0.93)

In [121]:
sub.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [132]:
subm[0].max()

0.93000001

In [134]:
submission.to_csv('2nd.gz', index=False, compression='gzip')

In [135]:
submission

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,2.680094e-08,8.238529e-09,8.865422e-07,3.089517e-07,9.470116e-11,2.809835e-07,5.128263e-06,8.705688e-09,1.003568e-08,...,6.057783e-09,6.015022e-09,8.140311e-09,1.651822e-09,2.712904e-10,1.498201e-08,1.261772e-10,1.240872e-10,1.023233e-07,3.402207e-09
1,00102ee9d8eb90812350685311fe5890,4.409035e-12,4.395454e-10,9.466591e-11,3.499728e-09,1.288064e-10,6.978071e-07,2.637466e-11,2.418920e-09,8.768275e-13,...,1.382480e-13,9.048941e-09,3.611206e-13,4.820020e-13,7.946374e-12,8.536657e-11,4.529498e-11,3.308734e-12,2.928139e-09,1.507963e-11
2,0012a730dfa437f5f3613fb75efcd4ce,1.441759e-07,6.407541e-10,1.719858e-06,1.056395e-09,5.077417e-01,2.035762e-08,7.594557e-08,8.651542e-07,3.030113e-08,...,2.485454e-11,5.811656e-09,9.306422e-06,2.945159e-07,9.096822e-08,2.985240e-09,4.609175e-08,1.343823e-07,8.142599e-09,1.794899e-08
3,001510bc8570bbeee98c8d80c8a95ec1,2.601991e-06,4.635281e-03,3.410056e-06,2.120565e-08,2.879387e-08,1.404188e-10,1.659390e-07,2.657265e-11,7.763093e-09,...,1.327743e-06,1.685799e-08,1.581223e-08,1.527565e-09,5.648541e-09,1.954087e-06,8.913732e-09,1.217109e-07,1.857164e-09,1.923937e-04
4,001a5f3114548acdefa3d4da05474c2e,2.367226e-04,1.720187e-06,7.717064e-07,1.403709e-03,6.398626e-10,1.526640e-06,1.890648e-04,2.548920e-07,5.499503e-08,...,9.028528e-04,2.481087e-04,8.593167e-09,1.573502e-08,1.239299e-08,4.512107e-05,5.253953e-06,6.223127e-09,4.301658e-05,1.946347e-02
5,00225dcd3e4d2410dd53239f95c0352f,9.529604e-07,6.793074e-08,4.006299e-07,1.207895e-09,4.491832e-12,1.961007e-09,6.978955e-10,3.988865e-13,2.587237e-12,...,1.916481e-11,3.116130e-11,1.778491e-10,3.371888e-10,2.102454e-09,1.680900e-09,7.825965e-10,5.232126e-09,1.602399e-10,2.993757e-09
6,002c2a3117c2193b4d26400ce431eebd,2.200322e-10,2.203210e-08,6.864013e-05,5.771421e-08,4.386780e-03,3.812013e-05,2.476081e-03,4.689917e-03,1.730167e-04,...,6.592745e-07,1.438501e-03,6.155725e-07,1.645831e-06,9.875382e-08,2.562226e-06,1.088021e-07,5.528710e-06,5.033983e-07,1.631207e-07
7,002c58d413a521ae8d1a5daeb35fc803,8.590322e-15,2.574094e-14,3.326684e-12,1.964292e-12,4.121560e-09,9.998488e-01,2.142084e-12,4.985143e-11,3.245525e-09,...,3.279725e-14,1.503878e-10,2.058827e-12,2.743715e-08,3.846519e-13,1.566222e-07,5.595840e-13,1.422465e-12,1.317602e-11,3.356998e-12
8,002f80396f1e3db687c5932d7978b196,1.773666e-08,2.653249e-05,6.920234e-08,2.468296e-11,3.398158e-06,4.444339e-09,4.147347e-09,3.035422e-07,6.956462e-04,...,8.258473e-08,3.580741e-07,6.535572e-11,9.177574e-06,9.855855e-09,2.854315e-01,5.938682e-07,3.857818e-06,3.587450e-09,4.549702e-08
9,0036c6bcec6031be9e62a257b1c3c442,1.960406e-06,6.401937e-13,1.050551e-11,2.069932e-12,1.566442e-11,4.917773e-10,1.965083e-09,1.222817e-10,1.307039e-12,...,9.802627e-14,4.580064e-08,3.697370e-13,5.945994e-09,6.416999e-13,2.221402e-09,6.467803e-11,4.837266e-10,1.177532e-07,1.300133e-10
